## Análise: resultado da ação proativa de Churn

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb.DuckDb import DuckDb
from Utils.DatasetProcessor.DatasetProcessorUtils import DatasetProcessorUtils
from Utils.DataAnalysis import DataAnalysis
from Utils import PandasNotebookConfigs
import project_config
from Utils.Datetime import DatetimeUtils
from Utils.Dict import DictUtils
from Utils.Number import NumberUtils

In [40]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.4f
Ajustando prints de linhas e colunas


In [ ]:
# 50% of the users will renew the plan
SUCESS_USER_PROACTIVE_ACTION = 0.50

COST_TO_KEEP_CUSTOMER = 200

RETAIN_MONTHS_QTY = 12

In [42]:
DUCK_DB = DuckDb()
DATA_TRANSFORMER = DataTransformer()
DATA_PROCESSOR_UTILS = DatasetProcessorUtils(DUCK_DB)
DATETIME_UTILS = DatetimeUtils()
DICT_UTILS = DictUtils()
DATA_ANALYSIS = DataAnalysis()
NUMBER_UTILS = NumberUtils()

In [43]:
df = DUCK_DB.load_table(
    project_config.PREDICTED_DATA_TABLE
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [44]:
len(df)

1251070

In [45]:
# Get only data to predict
df = df[df['no_churn_information'] == True]
len(df)

590951

In [46]:
# Get only users from the last month
max_safra = df['safra'].max()
print(f'Registros da safra de {max_safra}')

df = df[df['safra'] == max_safra]
len(df)

Registros da safra de 201702


173975

In [47]:
# users_df = df.groupby('msno').tail(1)
users_df = df.copy()

In [48]:
predicted_is_churn_df = users_df[users_df['predicted_is_churn'] == True]
predicted_not_is_churn_df = users_df[users_df['predicted_is_churn'] == False]

false_percentage = len(predicted_not_is_churn_df) * 100 / len(df)
true_percentage = 100 - false_percentage

print(f'Qtd. total de registros: {len(df)}')
print(f'Qtd. de previsões de churn: {len(predicted_is_churn_df)} ({NUMBER_UTILS.format_float_number(true_percentage, 2)}%)')
print(f'Qtd. de previsões de NÃO churn: {len(predicted_not_is_churn_df)} ({NUMBER_UTILS.format_float_number(false_percentage, 2)}%)')

Qtd. total de registros: 173975
Qtd. de previsões de churn: 44285 (25,45%)
Qtd. de previsões de NÃO churn: 129690 (74,54%)


In [49]:
mean_price = users_df['price_per_month'].mean()
print(f'Preço médio pago por mês: ${mean_price:.2f}')

Preço médio pago por mês: $132.32


In [50]:
proactive_action_gross_income = len(predicted_is_churn_df) * SUCESS_USER_PROACTIVE_ACTION * mean_price * RETAIN_MONTHS_QTY
costs = len(predicted_is_churn_df) * COST_TO_KEEP_CUSTOMER
liquid_income = proactive_action_gross_income - costs

print(f'Total de usuários afetados pela ação: {NUMBER_UTILS.format_integer_number(len(predicted_is_churn_df))}')

print(f'Valor total arrecadado pela ação proativa: ${NUMBER_UTILS.format_float_number(proactive_action_gross_income, 2)}')
print(f'Custos totais: ${NUMBER_UTILS.format_float_number(costs)}')
print(f'Total líquido: ${NUMBER_UTILS.format_float_number(liquid_income, 2)}')

Total de usuários afetados pela ação: 44.285
Valor total arrecadado pela ação proativa: $35.159.894,19
Custos totais: $8.857.000
Total líquido: $26.302.894,19
